In [1]:
import os
from dotenv import load_dotenv
from typing import TypedDict

load_dotenv()


True

In [10]:
from langgraph.graph import StateGraph, START, END
from langchain_deepseek import ChatDeepSeek
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [60]:
LLMS = {}

LLMS['gemini'] = ChatGoogleGenerativeAI(
    api_key = os.getenv("GOOGLE_API_KEY"),
    model = 'gemini-1.5-flash'
)

LLMS['deepseek'] = ChatDeepSeek(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    api_base=os.getenv("OPENROUTER_BASE_URL"),
    model="deepseek/deepseek-chat-v3-0324:free"
)

LLMS['moonshotai'] = ChatGroq(
    api_key = os.getenv("GROQ_API_KEY"),
    model = "moonshotai/kimi-k2-instruct"
)


In [61]:
llm = LLMS['moonshotai']


In [ ]:
class LLMState(TypedDict):
    query: str
    answer: str


In [ ]:

query_answer_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a helpful assistant. You help users to solve their query. You answer in a polite manner.
    Write like a confident, clear thinking human speaking to another smart human.
    Avoid robotic phrases like 'in today's fast-paced world', 'leveraging synergies', or 'furthermore'.
    Skip unnecessary dashes (-), quotation marks (“”), and corporate buzzwords like 'cutting-edge', 'robust', or 'seamless experience'.
    No Al tone. No fluff. No filler.
    Use natural transitions like 'here's the thing', 'let's break it down', or 'what this really means is...'
    Keep sentences varied in length and rhythm, like how real people speak or write.
    Prioritize clarity, personality, and usefulness. Every sentence should feel intentional, not generated.
    Sometimes try to use emoji to let user feel good.
    """),
    ("human", "{query}")
])


In [ ]:
def query_answer(state: LLMState) -> LLMState:
    query = state['query']
    chain = query_answer_prompt | llm | StrOutputParser()
    answer = chain.invoke(input={
        'query': query
    })
    state['answer'] = answer
    
    return state


In [ ]:
graph = StateGraph(LLMState)

graph.add_node('query_answer', query_answer)

graph.add_edge(START, 'query_answer')
graph.add_edge('query_answer', END)

workflow = graph.compile()


In [ ]:
final_state = workflow.invoke({
    'query': "Hey there dude!"
})
print(final_state)


{'query': 'Hey there dude!', 'answer': 'Yo! 😄 What’s on your mind today?'}
